# 作業 : 實做向量拼接方式 ATTENTION

# [作業目標]
- 實做向量拼接方式 ATTENTION
- 運用 實做的 ATTENTION FUNCTION 在之前的 RNN seq 2 seq attention

# [作業重點]
向量拼接方式 ATTENTION
- 先將 $q$ and $k$ concat 起來
- 然後經過一層 $W$ attention 自學參數,
- 和一個 $tanh$ activation function. 
- 最後乘以一個 $V_t$ 調整成一個同等於input seq 的數列.
$$
R(q,k)=v^Ttanh(W[q;k])
$$

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
# 請在這邊實做向量拼接方式 ATTENTION 
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_emb_dim):
        super().__init__()
        # (enc_hid_dim * 2) is from k, dec_emb_dim is from q
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_emb_dim, dec_emb_dim)
        self.v = nn.Linear(dec_emb_dim, 1, bias=False)
        
    def forward(self, dec_input, encoder_outputs, mask):
        # dec_input = [1, bz, dec_emb_dim]
        # encoder_outputs = [src len, bz, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        # repeat dec_input state src_len times
        # 這邊開始，同學可以跟隨我們的建議完成程式或是自行寫作
        # 整理代表 q and k 的 dec_input and encoder_output 
        dec_input = dec_input.permute(1,0,2).expand(batch_size, src_len, dec_input.size(2))
        encoder_outputs = encoder_outputs.permute(1,0,2)
        # dec_input = [bz, src len, dec_emb_dim]
        # encoder_outputs = [bz, src len, enc hid dim * 2]
        
        # 計算 向量拼接方式 ATTENTION
        # 先將 q and k concat 起來
        # 然後經過一層 W attention 自學參數,
        # 和一個 tanh activation function.
        # 最後乘以一個 Vt 調整成一個同等於input seq 的數列.
        attention = torch.tanh(self.attn(torch.cat([encoder_outputs, dec_input], dim=-1)))
        # attention = [bz, src len, dec_emb_dim]
        
        # 將 ATTENTION 結果乘以 V
        attention = torch.squeeze(self.v(attention), dim=-1)
        # attention = [bz, src len, 1] -> [bz, src len]
        
        # apply MASK 建議使用 tensor 的 masked_fill 
        attention = attention.masked_fill(mask==0, -1e10)
        
        return F.softmax(attention, dim = 1)